# Seq2Seq LSTM model - Translate and Predict

The encoder-decoder model provides a pattern for using recurrent neural networks to address challenging sequence-to-sequence prediction problems such as machine translation.

Encoder-decoder models can be developed in the Keras Python deep learning library and an example of a neural machine translation system developed with this model has been described on the Keras blog, with sample code distributed with the Keras project.

This example can provide the basis for developing encoder-decoder LSTM models for your own sequence-to-sequence prediction problems.

In [0]:
import numpy as np

In [0]:
np.random.seed(42)

## 1. Prepare data
Data for this exercise can be downloaded from http://www.manythings.org/anki/

### 1.1 Download and extract sentence pairs

In [0]:
!wget http://www.manythings.org/anki/hin-eng.zip

In [0]:
import zipfile
import io

In [0]:
zf = zipfile.ZipFile('hin-eng.zip', 'r')


In [0]:
data = ''
with zf.open('hin.txt') as readfile:
  for line in io.TextIOWrapper(readfile, 'utf-8'):
    data += line

In [0]:
data =  data.split('\n')

In [0]:
data[100:105]

Review the data

In [0]:
len(data)

### 1.2 Separate out Encoder and Decoder input data

In [0]:
encoder_text = []

In [0]:
decoder_text = []

In [0]:
for line in data:
    try:
        in_txt, out_txt = line.split('\t')
        encoder_text.append(in_txt)
        
        # Add tab '<start>' as 'start sequence in target
        # And '<end>' as End
        decoder_text.append('<start> ' + out_txt + ' <end>')
    except:
        pass #ignore data which goes into error        

In [0]:
decoder_text[100:105]

In [0]:
encoder_text[100:105]

### 1.3 Build Sequences for Encoder and Decoder Input

In [0]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

Encoder tokenizer

In [0]:
encoder_t = Tokenizer()

In [0]:
encoder_t.fit_on_texts(encoder_text)

In [0]:
encoder_seq = encoder_t.texts_to_sequences(encoder_text)

In [0]:
max_encoder_seq_length = max([len(txt) for txt in encoder_seq])

In [0]:
max_encoder_seq_length

In [0]:
encoder_vocab_size = len(encoder_t.word_index)

In [0]:
encoder_vocab_size

In [0]:
encoder_text[100:105]

In [0]:
encoder_seq[100:105]

Decoder tokenizer

In [0]:
decoder_t = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')

In [0]:
decoder_t.fit_on_texts(decoder_text)

In [0]:
decoder_seq = decoder_t.texts_to_sequences(decoder_text)

In [0]:
max_decoder_seq_length = max([len(txt) for txt in decoder_seq])

In [0]:
max_decoder_seq_length

In [0]:
decoder_vocab_size = len(decoder_t.word_index)

In [0]:
decoder_vocab_size

In [0]:
decoder_text[100:105]

In [0]:
decoder_seq[100:105]

### 1.4 Padding Sequences

In [0]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [0]:
encoder_input_data = pad_sequences(encoder_seq, maxlen=max_encoder_seq_length, padding='post')

In [0]:
decoder_input_data = pad_sequences(decoder_seq, maxlen=max_decoder_seq_length, padding='post')

In [0]:
encoder_input_data.shape

In [0]:
decoder_input_data.shape

Integer to Word converter for Decoder data

In [0]:
int_to_word_decoder = dict((i,c) for c, i in decoder_t.word_index.items())

In [0]:
int_to_word_decoder[15]

### 1.5 Building Decoder Output data

In [0]:
decoder_target_data = np.zeros((decoder_input_data.shape[0], decoder_input_data.shape[1]))

In [0]:
for i in range(decoder_input_data.shape[0]):
    for j in range(1,decoder_input_data.shape[1]):
        decoder_target_data[i][j-1] = decoder_input_data[i][j]        

In [0]:
decoder_input_data[0]

In [0]:
decoder_target_data[0]

Convert target data in one hot vector

In [0]:
from tensorflow.python.keras.utils import  to_categorical

In [0]:
decoder_target_one_hot = np.zeros((decoder_input_data.shape[0], 
                                   decoder_input_data.shape[1],
                                   len(decoder_t.word_index)+1))

In [0]:
for i in range(decoder_target_data.shape[0]):
    for j in range(decoder_target_data.shape[1]):
        decoder_target_one_hot[i][j] = to_categorical(decoder_target_data[i][j],
                                                      num_classes=len(
                                                          decoder_t.word_index)+1)    

In [0]:
decoder_target_one_hot.shape

## 2. Building the Training Model

In [0]:
from tensorflow.python.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.python.keras.callbacks import ModelCheckpoint

Define config parameters

In [0]:
encoder_embedding_size = 50
decoder_embedding_size = 50
rnn_units = 256

### 2.1 Build Encoder layers

In [0]:
encoder_inputs = Input(shape=(None,))

In [0]:
encoder_embedding = Embedding(encoder_vocab_size+1, encoder_embedding_size)

In [0]:
encoder_embedding_output = encoder_embedding(encoder_inputs)

In [0]:
x, state_h, state_c = LSTM(rnn_units,return_state=True)(encoder_embedding_output)

In [0]:
encoder_states = [state_h, state_c]

### 2.2 Build Decoder layers

In [0]:
decoder_inputs = Input(shape=(None,))

In [0]:
decoder_embedding = Embedding(decoder_vocab_size + 1, decoder_embedding_size)

In [0]:
decoder_embedding_output = decoder_embedding(decoder_inputs)

In [0]:
decoder_rnn = LSTM(rnn_units, return_sequences=True, return_state=True)

In [0]:
#Initialize initial state with encoder_states
#Output will be all hidden sequences, last 'h' state and last 'c' state
x,_,_ = decoder_rnn(decoder_embedding_output, initial_state=encoder_states)

In [0]:
decoder_dense = Dense(decoder_vocab_size + 1, activation='softmax')

In [0]:
decoder_outputs = decoder_dense(x)

### 2.3 Build Model using both Encoder and Decoder layers

In [0]:
from tensorflow.python.keras.models import Model

In [0]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

## 3. Train the model

In [0]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_one_hot,
          batch_size=64,
          epochs=1000,
          validation_split=0.2)

In [0]:
model.save('drive/AI-ML/models/seq2seq_eng_hin_training.h5')

## 4. Building Model for Prediction

### 4.1 Build the Encoder Model to predict Encoder States

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

### 4.2 Build the Decoder Model 

1. Define Input for both 'h' state and 'c' state initialization
2. Get RNN outputs along with h and c state
3. Define Decoder Output
4. Build Model

In [0]:
decoder_state_input_h = Input(shape=(rnn_units,))

In [0]:
decoder_state_input_c = Input(shape=(rnn_units,))

In [0]:
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

Get RNN outputs, state(s)

In [0]:
x = decoder_embedding(decoder_inputs)

In [0]:
#We will use the layer which we trained earlier
rnn_outputs, state_h, state_c = decoder_rnn(x, initial_state=decoder_states_inputs)

In [0]:
#Why do we need this?
decoder_states = [state_h, state_c]

get Decoder output

In [0]:
decoder_outputs = decoder_dense(rnn_outputs)

Build Decoder Model

In [0]:
decoder_model = Model([decoder_inputs] + decoder_states_inputs,  #Model inputs
                     [decoder_outputs] + decoder_states)

## 5.0 Predicting Output

Build a prediction function

In [0]:
def decode_sentence(input_sequence):
    
    #Get the encoder state values
    decoder_initial_states_value = encoder_model.predict(input_sequence)
    
    #Build a sequence with '<start>' - starting sequence for Decoder
    target_seq = np.zeros((1,1))    
    target_seq[0][0] = decoder_t.word_index['<start>']
    
    #flag to check if prediction should be stopped
    stop_loop = False
    
    #Initialize predicted sentence
    predicted_sentence = ''
    
    #start the loop
    while not stop_loop:
        
        predicted_outputs, h, c = decoder_model.predict([target_seq] + 
                                                        decoder_initial_states_value)
        
        #Get the predicted output with highest probability
        predicted_output = np.argmax(predicted_outputs[0,-1,:])
        
        #Get the predicted word from predicter integer
        predicted_word = int_to_word_decoder[predicted_output]
        
        #Check if prediction should stop
        if(predicted_word == '<end>' or len(predicted_sentence) > max_decoder_seq_length):
            
            stop_loop = True
            continue
                    
        #Updated predicted sentence
        if (len(predicted_sentence) == 0):
            predicted_sentence = predicted_word
        else:
            predicted_sentence = predicted_sentence + ' ' + predicted_word
            
        #Update target_seq to be the predicted word index
        target_seq[0][0] = predicted_output
        
        #Update initial states value for decoder
        decoder_initial_states_value = [h,c]
        
    
    return predicted_sentence

Call Prediction function above

In [0]:
#Get a random sentence
start_num = np.random.randint(0, high=len(encoder_text) - 10)
print(start_num)

for i in range(start_num, start_num + 10):
    input_seq = encoder_input_data[i : i+1]
    predicted_sentence = decode_sentence(input_seq)
    print('--------')
    print ('Input sentence: ', encoder_text[i])
    print ('Predicted sentence: ', predicted_sentence )

## 6. Save Prediction models and tokenizers 

In [0]:
#Save encoder and decoder model for Prediction
encoder_model.compile(optimizer='adam', loss='mse')
decoder_model.compile(optimizer='adam', loss='categorical_crossentropy')
encoder_model.save('drive/AI-ML/models/seq2seq_encoder_eng_hin.hd5')
decoder_model.save('drive/AI-ML/models/seq2seq_decoder_eng_hin.hd5')

In [0]:
#Save tokenizers
import pickle

pickle.dump(encoder_t,open('drive/AI-ML/models/encoder_tokenizer_eng','wb'))
pickle.dump(decoder_t,open('drive/AI-ML/models/decoder_tokenizer_hin','wb'))